In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt


from qiskit import Aer, execute

from qiskit_ionq_provider import IonQProvider 

#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')

In [2]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7f463b426090>()>,
 <IonQQPUBackend('ionq_qpu') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7f463b426090>()>]

In [12]:
from qiskit import QuantumCircuit
from random import randint

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate

def dec_to_bin(x,d):
    output = str(int(bin(x)[2:]))
    if len(output) < d-1:
        zeros = ("0" * (d-1-len(output)))
        return zeros + output
    elif len(output) > d-1:
        print("input number needed to be cut to fit parameters")
        return output[len(output)-d-1:len(output)]
    else:
        return output

def crypto_hash_init(input_num, K):
    
    
    d = len(K)
    bitstring = dec_to_bin(input_num,d)
    n = len(bitstring)
    big_N = 2**n
    print(n)
    
    

    qr = QuantumRegister(d)
    qc_hash = QuantumCircuit(qr)
    
    return (qc_hash, qr, d, bitstring, n, big_N)
    

In [26]:
def crypto_hash_add_H(qc_hash, qr, d, bitstring, n, big_N):
    for i in range(d-1):
        qc_hash.h(i)
    
    return qc_hash

In [30]:
def crypto_hash_final(qc_hash, qr, d, bitstring, n, big_N, K):
    print(bitstring)
    print(n)
    for n_index in range(1,n+1):
        control_states = "0" * (d-1)
        if int(bitstring[n_index-1]) == 1:
            for perm in range(0,2**(d-1)):     
                control_states = dec_to_bin(perm,d)
                print(control_states)
                print(d)
                i = sum(int(c) for c in control_states.strip())
                Y_Rotate_Gate = RYGate(4*np.pi*K[i-1]*2**n_index/big_N).control(d-1, ctrl_state=control_states)
                qc_hash.compose(Y_Rotate_Gate, qr, inplace=True)
        
    return qc_hash


In [31]:
def crypto_hash(input_num, K):
    (qc_hash, qr, d, bitstring, n, big_N) = crypto_hash_init(input_num, K)
    qc = crypto_hash_add_H(qc_hash, qr, d, bitstring, n, big_N)
    qc_hash = crypto_hash_final(qc, qr, d, bitstring, n, big_N, K)
    
    backend = Aer.get_backend('statevector_simulator')
    state = execute(qc,backend).result().get_statevector()
    return(state)



In [32]:
crypto_hash(3,[1,2,4,1,7,1])

5
00011
5
00000
6
00001
6
00010
6
00011
6
00100
6
00101
6
00110
6
00111
6
01000
6
01001
6
01010
6
01011
6
01100
6
01101
6
01110
6
01111
6
10000
6
10001
6
10010
6
10011
6
10100
6
10101
6
10110
6
10111
6
11000
6
11001
6
11010
6
11011
6
11100
6
11101
6
11110
6
11111
6
00000
6
00001
6
00010
6
00011
6
00100
6
00101
6
00110
6
00111
6
01000
6
01001
6
01010
6
01011
6
01100
6
01101
6
01110
6
01111
6
10000
6
10001
6
10010
6
10011
6
10100
6
10101
6
10110
6
10111
6
11000
6
11001
6
11010
6
11011
6
11100
6
11101
6
11110
6
11111
6


array([-1.76776695e-01+0.j, -1.76776695e-01+0.j, -1.76776695e-01+0.j,
        1.76776695e-01+0.j, -1.76776695e-01+0.j,  1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j, -1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j,  1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j,  1.76776695e-01+0.j,
       -1.76776695e-01+0.j, -1.76776695e-01+0.j,  1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j,  1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j, -1.76776695e-01+0.j,
        1.76776695e-01+0.j,  1.76776695e-01+0.j,  1.76776695e-01+0.j,
       -1.76776695e-01+0.j,  1.76776695e-01+0.j, -1.76776695e-01+0.j,
       -1.76776695e-01+0.j, -1.76776695e-01+0.j, -1.60273192e-17+0.j,
       -1.91866412e-16+0.j, -2.29662131e-17+0.j,  1.64272176e-16+0.j,
        4.30984643e-16+0.j, -1.52030184e-16+0.j, -1.20258467e-16+0.j,
       -5.53320655e-16+0.j,  3.12144758e-16+0.j, -5.06721945e-17+0.j,
        4.14395037e-